<a href="https://colab.research.google.com/github/trefftzc/partition/blob/main/partition_mpi4py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving the partition problem with mpi4py

In [1]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.2/466.2 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.1-cp310-cp310-linux_x86_64.whl size=4266352 sha256=f981a09027625d82644bedff4064575ccbe53e76de2ed09fde534b2072a30890
  Stored in directory: /root/.cache/pip/wheels/3c/ca/13/13218a83854023ccec184e3af482f0f038b434aa32c19afee8
Successfully built mpi4py


In [2]:
%%writefile partition_mpi4py.py
from mpi4py import MPI
import time
import numpy as np



# This function has not changed from the previous versions
# There are no changes related to MPI in this function
def evaluatePartition(value,n,array):
  sum0s = 0
  sum1s = 0
  mask = 1
  for i in range(0,n):
    if ((mask & value) != 0):
      sum1s = sum1s + array[i]
    else:
      sum0s = sum0s + array[i]
    mask = mask*2
  if (sum0s == sum1s):
    return 1
  else:
    return 0



# This function has not changed from the previous versions
# There are no changes related to MPI in this function
def printResults(value,n,array):
  print("Solution:")
  print("First partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) != 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)

  print("Second partition: ")
  mask = 1
  sum = 0
  for i in range(0,n):
    if ((mask & value) == 0):
      print(array[i],end=" ")
      sum = sum + array[i]
    mask = mask * 2

  print("sum: ",sum)





def solve(numpyArray,nPartitions,n,begin,end):
  solutionFound = 0
  for i in range(begin,end):
    r = evaluatePartition(i,n,numpyArray)
    if (r==1):
      solutionFound = max(solutionFound,i)
  if (solutionFound != 0):
    return solutionFound
  else:
    return -1



# The changes necessary to use MPI (mpi4py)
# are limited to the main program
if __name__ == "__main__":
# For all nodes
# Find their rank
# Find the number of nodes being used in this parallel program
  comm = MPI.COMM_WORLD
  rank = comm.Get_rank()
  number_nodes = comm.Get_size()
  # If this is the master node, read the problem
  if rank == 0:
    start = time.time()

  if rank == 0:
    n = int(input())
    values = []
    for i in range(n):
      values.append( int(input()) )

  # Print the instance of the problem
    print("Problem size: ",n)
    print("Problem instance: ",values)
  else:
    n = None

# Broadcast the size of the problem, the number of entries
# in the multiset to all participating nodes
  n = comm.bcast(n,root=0)
# Broadcast the multiset to all participating nodes
  if rank == 0:
    npValues = np.array(values)
  else:
    npValues = np.arange(n,dtype='i')
  npValues = comm.bcast(npValues,root=0)
  # print("npValues: ",npValues)
  # Calculate the size of the power set

  nPartitions = int ( 2 ** n )

# Calculate the portion of the power set that each
# node will be working on

  portionEachNode = nPartitions // number_nodes
  initial = portionEachNode * rank;

  if (rank != (number_nodes-1)):
    final = initial + portionEachNode
  else:
    final = nPartitions
  max_in_this_node = solve(npValues,nPartitions,n,initial,final)
# Reduce across all nodes to find the max across all nodes
  result = comm.reduce(max_in_this_node,op=MPI.MAX)

  if rank == 0:
    if result == -1:
      print("No solution found.")
    else:
      printResults(result,n,npValues)

  if rank == 0:
    end = time.time()
    elapsed = end - start
    print("The program took: ",elapsed," seconds.")

Writing partition_mpi4py.py


In [5]:
%%writefile test11.Text
11
1
2
3
4
5
6
7
8
9
10
11

Overwriting test11.Text


In [6]:
!OMPI_ALLOW_RUN_AS_ROOT=1
!mpiexec --allow-run-as-root -n 2 --oversubscribe python partition_mpi4py.py < test11.Text

Problem size:  11
Problem instance:  [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
Solution:
First partition: 
3 9 10 11 sum:  33
Second partition: 
1 2 4 5 6 7 8 sum:  33
The program took:  0.06608128547668457  seconds.
